In [2]:
from __future__ import absolute_import, division, print_function

# Elevation & Length of Day

To account for the amount of sunlight avaialable to the plants at the different locations and times, get the length of the day. Here, I do this by using the 'Astral' package (https://pythonhosted.org/astral/index.html .

I also leverage the Google Elevation API to retrieve the elevation for each location.

The results are saved (and pickled) in dictionaries for easy lookup.

## Imports

In [3]:
import os
import pickle
import time
import requests 
import json
import datetime

import numpy as np
import pandas as pd

from astral import Location

## Data

In [4]:
cwd = os.getcwd()
data = os.path.join(cwd,'data','wheat-2013-supervised.csv')
df_2013 = pd.read_csv(data)
data = os.path.join(cwd,'data','wheat-2014-supervised.csv')
df_2014 = pd.read_csv(data)


In [5]:
df_2013['Date'].max()

'6/3/2014 0:00'

In [6]:
df_2014['Date'].max()

'6/3/2015 0:00'

## Locations

In [7]:
df_2013['Location'] = list(zip(df_2013['Longitude'], df_2013['Latitude']))
locs_2013 = df_2013['Location'].unique().tolist()
len(locs_2013)

1014

In [8]:
df_2014['Location'] = list(zip(df_2014['Longitude'], df_2014['Latitude']))
locs_2014 = df_2014['Location'].unique().tolist()
len(locs_2014)

1035

In [9]:
# Union of locations without repetitions
elevation = {}
for loc in locs_2013:
    elevation[loc] = 0
for loc in locs_2014:
    if loc in elevation:
        pass
    else:
        elevation[loc] = 0

In [10]:
print('Number of unique locations across 2013 and 2014: {}'.format(len(elevation.keys())))

Number of unique locations across 2013 and 2014: 1167


## Elevation

In [11]:
# Get API key from system variable
google_api_key = os.environ['GOOGLE_API_KEY']

In [12]:
run_API_query = False

base_string = 'https://maps.googleapis.com/maps/api/elevation/json?locations='
locs = elevation.keys()

if run_API_query is True:
    for idx, loc in enumerate(locs):
        lat_string = str(loc[1])
        lng_string = str(loc[0])
        call_string = base_string + lat_string + ',' + lng_string + '&key=' + google_api_key
        if idx%10 == 0: 
            print(idx)
        if elevation[loc] == 0:
            response = requests.get(call_string)
            try:
                tmp_elevation = response.json()['results'][0]['elevation']
                elevation[loc] = tmp_elevation
            except:
                print('Something went wrong', lat_string, lng_string)
                print(response.json())
            

else:
    # INSERT
    #
    # Load data saved at end of this notebook.
    # Can be used to experiment with the Astral code below without
    # having to re-run the Google API queries above.

    with open(os.path.join('data','elevation.pickle'), 'rb') as handle:
        elevation = pickle.load(handle)

    


            

## Length of day

Use the 'Astral' package (https://pythonhosted.org/astral/index.html) to calculate the length of the day (~ hours of sunlight) for each location.

In [13]:
locs = elevation.keys()
length_of_day = {}
for idx, loc in enumerate(locs):
    # Initialize location
    l = Location()
    # Set attributes
    l.name = ''
    l.region = ''
    l.latitude = loc[1]
    l.longitude = loc[0]
    l.timezone = 'UTC'
    l.elevation = elevation[loc]
    sun = l.sun(date=datetime.date(2014, 3, 3),local=True)
    td = sun['sunset'] - sun['sunrise']
    tmp_lod = td.total_seconds() / 3600. 
    length_of_day[loc] = tmp_lod 


## Save data

In [14]:
# Store data (serialize)
with open(os.path.join('data','elevation.pickle'), 'wb') as handle:
    pickle.dump(elevation, handle)

with open(os.path.join('data','length_of_day.pickle'), 'wb') as handle:
    pickle.dump(length_of_day, handle)

